In [113]:
#0918:all question and column links from topic "贺建奎" 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from pyquery import PyQuery as pq
import pandas as pd
from pandas import Series, DataFrame
import csv
import urllib.request
import datetime
import time
import re

#------------------------------------------------------------1st stage,collect all question/column links
driver = webdriver.Chrome()
url = "https://www.zhihu.com/topic/20690485/hot"
    
driver.get(url)
driver.implicitly_wait(10)

time.sleep(2)

#scroll to the topic bottom
for i in range(20):
    js = "var q = document.documentElement.scrollTop="+str(i*1000)
    driver.execute_script(js)
    time.sleep(1)
time.sleep(2)

listql = []
soup = BeautifulSoup(driver.page_source,"lxml")
for b in soup.findAll("meta",attrs = {"itemprop":"url"}):
    c = b["content"]
    if "question" in c and "answer" not in c and c not in listql:
        listql.append(c) 
    if "zhuanlan" in c and c not in listql:
        listql.append(c) 

#url = "https://www.zhihu.com/topic/20741067/top-answers"
driver.find_element_by_link_text("精华").click()
time.sleep(2)
driver.implicitly_wait(10)
time.sleep(2)

#scroll to the topic bottom
for i in range(5):
    js = "var q = document.documentElement.scrollTop="+str(i*1000)
    driver.execute_script(js)
    time.sleep(1)
time.sleep(2)

soup = BeautifulSoup(driver.page_source,"lxml")
for b in soup.findAll("meta",attrs = {"itemprop":"url"}):
    c = b["content"]
    if "question" in c and "answer" not in c and c not in listql:
        listql.append(c) 
    if "zhuanlan" in c and c not in listql:
        listql.append(c) 

        
#url = "https://www.zhihu.com/topic/20741067/unanswered"
driver.find_element_by_link_text("等待回答").click()
driver.implicitly_wait(10)
time.sleep(2)

#scroll to the topic bottom
for i in range(4):
    js = "var q = document.documentElement.scrollTop="+str(i*1000)
    driver.execute_script(js)
    time.sleep(1)
time.sleep(2)

soup = BeautifulSoup(driver.page_source,"lxml")
for b in soup.findAll("a"):
    c = b["href"]
    d = "https://www.zhihu.com" + c
    if "question" in c and len(c) == 19 and "topic" not in c and "write_answer" not in c and d not in listql:
        listql.append(d)

listql = [re.sub("^https:/+","https://",i) for i in listql]

save = "/Users/CM/desktop/ScrapeZQ2/QClink/ql-hjk.csv"
df = pd.DataFrame(listql,columns = ["QClink"])
df1 = df.to_csv(save, index = False)
print(pd.read_csv(save))
#------------------------------------------------------------2nd stage,collect question/column details
driver.quit()

                                      QClink
0      https://zhuanlan.zhihu.com/p/55099601
1   https://www.zhihu.com/question/303774068
2      https://zhuanlan.zhihu.com/p/62712684
3      https://zhuanlan.zhihu.com/p/71253856
4      https://zhuanlan.zhihu.com/p/60834959
5      https://zhuanlan.zhihu.com/p/55505292
6   https://www.zhihu.com/question/309940522
7      https://zhuanlan.zhihu.com/p/66865251
8      https://zhuanlan.zhihu.com/p/57388447
9      https://zhuanlan.zhihu.com/p/51596043
10  https://www.zhihu.com/question/313198233
11     https://zhuanlan.zhihu.com/p/56530809
12     https://zhuanlan.zhihu.com/p/55485498
13     https://zhuanlan.zhihu.com/p/55541144
14     https://zhuanlan.zhihu.com/p/56906988
15     https://zhuanlan.zhihu.com/p/59392764
16  https://www.zhihu.com/question/327791758
17     https://zhuanlan.zhihu.com/p/55469985
18     https://zhuanlan.zhihu.com/p/58813491
19     https://zhuanlan.zhihu.com/p/53914031
20  https://www.zhihu.com/question/307655442
21  https:

In [115]:
#0918:question and column detail from topic "crispr" 
driver = webdriver.Chrome()

#scan the QR code manually to login
driver.get("https://www.zhihu.com/")
time.sleep(12)

with open("/Users/CM/desktop/ScrapeZQ2/qca-hjk.csv","w",encoding = "utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["column","title","flwr","v","authlink","upno","cmtno","andate","textlen","imgno","antext"])
    df1 = pd.read_csv("/Users/CM/desktop/ScrapeZQ2/QClink/ql-hjk.csv")["QClink"].values

    for qcl in df1:
        driver.get(qcl)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,"lxml")
    
        try:            
            if "question" in qcl:
                #unfold qtext
                time.sleep(1)
                try:
                    driver.find_element_by_xpath("//div[@class='QuestionRichText QuestionRichText--expandable QuestionRichText--collapsed']").click()
                    time.sleep(1)
                except:
                    time.sleep(1)
                
                #get question info
                column = "0"
                qtitle = soup.find("h1", class_="QuestionHeader-title").get_text()+soup.find("div", class_="QuestionHeader-detail").get_text()
                qflwr = soup.findAll("strong", class_="NumberBoard-itemValue")[0].get_text()
                qv = soup.findAll("strong", class_="NumberBoard-itemValue")[1].get_text()                

                if len(soup.findAll("h4", class_="List-headerText")) > 0:                   
                    #scroll to the bottom
                    xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                    an = soup.findAll("span", class_="RichText ztext CopyrightRichText-richText")
                    if len(an) > 0:
                        n = 1
                        while len(xhd) == 0:
                            js = "var q = document.documentElement.scrollTop="+str(n*1000)
                            driver.execute_script(js)
                            soup = BeautifulSoup(driver.page_source,"lxml")
                            xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                            n+=1
                            time.sleep(0.6)
                    
                    #get all answer info
                    ans = soup.findAll("div", class_="List-item")
                    for ai in ans:
                        try:
                            authlink = re.sub("^//","https://",ai.find("a", class_="UserLink-link")["href"])
                        except:
                            authlink = "AnonymousUser"
                        upno = ai.find("button", class_="Button VoteButton VoteButton--up").get_text()
                        cmtno = ai.find("button", class_="Button ContentItem-action Button--plain Button--withIcon Button--withLabel").get_text()
                        andate = ai.find("div", class_="ContentItem-time").get_text()
                        imgno = str(len(ai.findAll("figure")))+"个图片"
                        antext = ai.find("span", class_="RichText ztext CopyrightRichText-richText").get_text()
                        textlen = str(len(antext))+"个文字"
                        writer.writerow([column,qtitle,qflwr,qv,authlink,upno,cmtno,andate,textlen,imgno,antext])
                else:
                    authlink = "AnonymousUser"
                    writer.writerow([column,qtitle,qflwr,qv,authlink])                   
                    
            elif "zhuanlan" in qcl:
                 
                column = "1"
                coltitle = soup.find("h1", class_="Post-Title").get_text()
                colflwr = "NoColflwr"
                colv = "NoColv"
                authlink = re.sub("^//","https://",soup.find("a", class_="UserLink-link")["href"])
                upno = soup.find("button", class_="Button VoteButton VoteButton--up").get_text()
                cmtno = soup.find("button", class_="Button BottomActions-CommentBtn Button--plain Button--withIcon Button--withLabel").get_text()
                coldate = soup.find("div", class_="ContentItem-time").get_text()
                imgno = str(len(soup.findAll("figure")))+"个图片"
                coltext = soup.find("div", class_="Post-RichTextContainer").get_text()
                textlen = str(len(coltext))+"个文字"
                writer.writerow([column,coltitle,colflwr,colv,authlink,upno,cmtno,coldate,textlen,imgno,coltext])
            time.sleep(2)
            
        except:
            print(qcl) 
            #fill up manually

print("done.")
time.sleep(1)
driver.quit()

https://www.zhihu.com/question/327791758
https://www.zhihu.com/question/317091956
done.


In [121]:
#0918 question error page processing
qlerr = ["https://www.zhihu.com/question/317091956"]

driver = webdriver.Chrome()
#scan the QR code manually to login
#driver.get("https://www.zhihu.com/")
#time.sleep(8)

with open("/Users/CM/desktop/ScrapeZQ2/qca-hjk1.csv","w",encoding = "utf-8") as f:
        writer = csv.writer(f)
        for ql in qlerr:
            
            driver.get(ql)
            #time.sleep(8)
            soup = BeautifulSoup(driver.page_source,"lxml")
            
            try:
                
            
                #unfold qtext
                time.sleep(1)
                try:
                    driver.find_element_by_xpath("//div[@class='QuestionRichText QuestionRichText--expandable QuestionRichText--collapsed']").click()
                    time.sleep(1)
                except:
                    time.sleep(1)
                
                #get question info
                column = "0"
                qtitle = soup.find("h1", class_="QuestionHeader-title").get_text()+soup.find("div", class_="QuestionHeader-detail").get_text()
                qflwr = soup.findAll("strong", class_="NumberBoard-itemValue")[0].get_text()
                qv = soup.findAll("strong", class_="NumberBoard-itemValue")[1].get_text()                
    
                if len(soup.findAll("h4", class_="List-headerText")) > 0:                   
                    #scroll to the bottom
                    xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                    an = soup.findAll("span", class_="RichText ztext CopyrightRichText-richText")
                    if len(an) > 0:
                        n = 1
                        while len(xhd) == 0:
                            js = "var q = document.documentElement.scrollTop="+str(n*1000)
                            driver.execute_script(js)
                            soup = BeautifulSoup(driver.page_source,"lxml")
                            xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                            n+=1
                            time.sleep(0.6)
                    
                    #get all answer info
                    soup = BeautifulSoup(driver.page_source,"lxml")
                    ans = soup.findAll("div", class_="List-item")
                    for ai in ans:
                        try:
                            authlink = re.sub("^//","https://",ai.find("a", class_="UserLink-link")["href"])
                        except:
                            authlink = "AnonymousUser"
                        upno = ai.find("button", class_="Button VoteButton VoteButton--up").get_text()
                        cmtno = ai.find("button", class_="Button ContentItem-action Button--plain Button--withIcon Button--withLabel").get_text()
                        andate = ai.find("div", class_="ContentItem-time").get_text()
                        imgno = str(len(ai.findAll("figure")))+"个图片"
                        antext = ai.find("span", class_="RichText ztext CopyrightRichText-richText").get_text()
                        textlen = str(len(antext))+"个文字"
                        writer.writerow([column,qtitle,qflwr,qv,authlink,upno,cmtno,andate,textlen,imgno,antext])
                else:
                    authlink = "AnonymousUser"
                    writer.writerow([column,qtitle,qflwr,qv,authlink])  
            except:
                print(ql)


    
print("done.")
driver.quit()

done.


In [124]:
#0918:answerers detail in topic "贺建奎" 
driver = webdriver.Chrome()

#scan the QR code manually to login
driver.get("https://www.zhihu.com/")
time.sleep(10)

with open("/Users/CM/desktop/ScrapeZQ2/ai-hjk.csv","w",encoding = "utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["authlink","authname","verified","authflwr","authupno"])
    df2 = pd.read_csv("/Users/CM/desktop/ScrapeZQ2/qca-hjk.csv")["authlink"].values
    for al in df2:
        if al != "AnonymousUser":
            driver.get(al)
            driver.implicitly_wait(8)
            soup = BeautifulSoup(driver.page_source,"lxml")
            
            try:
                if len(soup.findAll("div", class_="ProfileMainPrivacy-mainContentWrapper")) == 0:
                    authlink = al
                    authname = soup.find("span",class_="ProfileHeader-name").get_text()
                    if soup.find("div", class_="Card-headerText").get_text() == "个人成就":
                        verified = "0"
                    if soup.find("div", class_="Card-headerText").get_text() == "认证与成就":
                        verified = "1"
                    authflwr = soup.findAll("strong", class_="NumberBoard-itemValue")[1].get_text()
                    for i in soup.findAll("div", class_="IconGraf"):
                        if "赞同" in i.get_text():
                            authupno = i.get_text()
                    writer.writerow([authlink,authname,verified,authflwr,authupno])
                    time.sleep(1)
            except:
                print(al)
print("done.")
            
time.sleep(1)
driver.quit()

done.


In [120]:
len("基因编辑是针对胚胎进行操作的，但胚胎的每个细胞的编辑效果是不一样的，结果是一个人的胚胎里就出现了各种不同基因的细胞。这种情况被称为嵌合体（chimera）。嵌合体的另一种翻译就是奇美拉。")

93